# Tutorial on Image Clustering

## Part 2

by Joris Guérin

## Import utils

In [ ]:
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import fowlkes_mallows_score as FM
from sklearn.metrics import normalized_mutual_info_score as NMI

import pickle
import cv2

from copy import deepcopy

In [ ]:
def confusion_matrix(clusters, true_labels):
    new_tl = deepcopy(true_labels)
    l = list(set(true_labels))
    for i in range(len(true_labels)):
        for j in range(len(l)):
            if true_labels[i] == l[j]:
                new_tl[i] = j
                
    conf_mat = np.zeros([len(set(clusters)), len(set(new_tl))])
    for i in range(len(clusters)):
        conf_mat[clusters[i], new_tl[i]] += 1

    return conf_mat

def purity(clusters, true_labels):
    conf_mat = confusion_matrix(clusters, true_labels)
    sum_clu  = np.max(conf_mat, axis = 1)
    sum_tot  = np.sum(sum_clu)

    pur = sum_tot / len(clusters)

    return pur

In [ ]:
im_path, lab_path = "./umist/raw", "./umist"
n_im = 575

fi = open("%s/true_labels.txt" % lab_path, "r")
true_labels = fi.readlines()
true_labels = np.array([int(lab.rstrip("\n")) for lab in true_labels])
fi.close()

raw_data = []
for i in range(n_im):
    raw_data.append(cv2.imread("%s/%s.png" % (im_path, i), cv2.IMREAD_GRAYSCALE).flatten())
raw_data = np.array(raw_data)

## Dimensionality reduction for data visualization

### t-SNE on raw images

In [ ]:
from sklearn.manifold import TSNE

tsne2D = TSNE(n_components = 2)
rawData2D = tsne2D.fit_transform(raw_data)

In [ ]:
plt.scatter(rawData2D[:, 0], rawData2D[:, 1])
plt.show()

In [ ]:
plt.scatter(rawData2D[:, 0], rawData2D[:, 1], c = true_labels, cmap = plt.get_cmap("tab20b"))
plt.show()

### t-SNE on VGG19 features

In [ ]:
feat_path = "./umist/cnn_features/"

feat_file = open(feat_path + "vgg19_fc2" + ".p", "rb")
feat_vgg19 = pickle.load(feat_file)
feat_file.close()

feat_vgg19_2D = tsne2D.fit_transform(feat_vgg19) 

plt.scatter(feat_vgg19_2D[:, 0], feat_vgg19_2D[:, 1], c = true_labels, cmap = plt.get_cmap("tab20b"))
plt.show()

## Ensemble clustering with multiple CNN feature extractors

In [ ]:
nets_list = [
    "densenet121_avg_pool",
    "densenet169_avg_pool",
    "densenet201_avg_pool",
    "inceptionV3_avg_pool",
    "inception_resnetV2_avg_pool",
    "nasnet_global_average_pooling2d_1",
    "resnet50_avg_pool",
    "vgg16_fc2",
    "vgg19_fc2",
    "xception_avg_pool"
]

agg20 = AgglomerativeClustering(20)

### Read data

In [ ]:
feat_path = "./umist/cnn_features/"

data = []
for net in nets_list:
    feat_file = open(feat_path + net + ".p", "rb")
    data.append(pickle.load(feat_file))
    feat_file.close()

    print(data[-1].shape)  

### Solve for each feature extractor

In [ ]:
preds = []
for d in data:
    preds.append(agg20.fit_predict(d))
    print(purity(preds[-1], true_labels), NMI(preds[-1], true_labels), FM(preds[-1], true_labels))
preds = np.array(preds)

### Group predictions

In [ ]:
import numba as nb

agg_ens = AgglomerativeClustering(20, affinity='precomputed', linkage="average")

@nb.jit
def computeM(P):
    n = len(P[0])
    M = np.zeros([n, n])
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            for k in range(len(P)):
                M[i, j] += int(P[k][i] == P[k][j])
            M[i, j] /= len(P)
    return M

def getFinalScores(Partition):
    M = computeM(Partition)

    clusters = agg_ens.fit_predict(-M)

    nmi = NMI(clusters, true_labels)
    pur = purity(clusters, true_labels)
    fm = FM(clusters, true_labels)
    return pur, nmi, fm

### Results

In [ ]:
final_scores = getFinalScores(preds)
print(final_scores)

purs = [purity(preds[i, :], true_labels) for i in range(10)]
nmis = [NMI(preds[i, :], true_labels) for i in range(10)]
fms = [FM(preds[i, :], true_labels) for i in range(10)]

purs.append(0)
purs.append(final_scores[0])
nmis.append(0)
nmis.append(final_scores[1])
fms.append(0)
fms.append(final_scores[2])

print("Purity scores")
fig = plt.figure()
ax = fig.add_subplot(111)
rects1 = ax.bar(range(len(purs)), purs, 0.4)
ax.set_ylim(bottom = 0.5)
plt.show()

print("NMI scores")
fig = plt.figure()
ax = fig.add_subplot(111)
rects1 = ax.bar(range(len(nmis)), nmis, 0.4)
ax.set_ylim(bottom = 0.65)
plt.show()

print("FM scores")
fig = plt.figure()
ax = fig.add_subplot(111)
rects1 = ax.bar(range(len(fms)), fms, 0.4)
ax.set_ylim(bottom = 0.35)
plt.show()

## JULE

In [ ]:
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Dense, BatchNormalization, Activation, Input, concatenate
from keras.regularizers import l2
from keras import backend as K

from utils import *

### Generating data

In [ ]:
cm = "tab20b"

n1 = 50
mu1 = np.array([2,5])
sig1 = np.array([[3,0],[0,2]])
groupe1 = np.random.multivariate_normal(mu1, sig1, n1)

n2 = 50
mu2 = np.array([7,10])
sig2 = np.array([[1,0],[0,2]])
groupe2 = np.random.multivariate_normal(mu2, sig2, n2)

n3 = 50
mu3 = np.array([9,2])
sig3 = np.array([[3,0],[0,3]])
groupe3 = np.random.multivariate_normal(mu3, sig3, n3)

data = np.r_[groupe1, groupe2, groupe3]
labels = np.array([0] * n1 + [1] * n2 + [2] * n3)

plt.scatter(data[:, 0], data[:, 1], c=labels, cmap = cm)
plt.show()
plt.scatter(data[:, 0], data[:, 1], cmap = cm)
plt.show()

agg3 = AgglomerativeClustering(3)
clu_agg = agg3.fit_predict(data)

print(NMI(labels, clu_agg))

### Parameters

In [ ]:
n_neighbors_pts = 20
n_neighbors_clu = 5
hlayer_shape = 100
reg_weight = 0.01
lrate = 0.001
n_neg = 10
unfold_rate = 0.9
n_clusters_target = 3
n_epochs = 20

### Initialize clusters

In [ ]:
_, indices = get_NearestNeighbors(data, n_neighbors_pts)
labels_cur = []
labels_cur.append(0)
labels_cur_table, labels_cur[-1] = initialize_clusters(indices, data.shape[0])
#print(len(labels_cur))
#print(sum([len(labels_cur_table[i]) for i in range(len(labels_cur_table))]))
n_clusters = len(list(set(labels_cur[-1])))
print(n_clusters)
plt.scatter(data[:, 0], data[:, 1], c=labels_cur[-1], cmap = cm)
plt.show()

### Initialize network

In [ ]:
network = Sequential()
network.add(Dense(hlayer_shape, input_shape=(2,), activation='relu'))
network.add(Dense(2, activation='linear', kernel_regularizer=l2(reg_weight)))

data_cur = []
data_cur.append(0)
data_cur[-1] = network.predict(data)
plt.scatter(data_cur[-1][:, 0], data_cur[-1][:, 1], c=labels_cur[-1], cmap = cm)
plt.savefig('initial_network.pdf', format = 'pdf')
plt.show()

anchor_in = Input(shape=(2,))
pos_in = Input(shape=(2, ))
neg_in = Input(shape=(2, ))

anchor_out = network(anchor_in)
pos_out = network(pos_in)
neg_out = network(neg_in)
merged_vector = concatenate([anchor_out, pos_out, neg_out], axis=-1)

trainable_model = Model(inputs=[anchor_in, pos_in, neg_in], outputs=merged_vector)
optimizer = Adam(lr = lrate)
trainable_model.compile(optimizer=optimizer, loss=triplet_loss)

### Train network

In [ ]:
for i in range(n_epochs):
    a, p, n = make_triplets(labels_cur[-1], n_clusters, n_neg)
    inputs = [data[a], data[p], data[n]]
    trainable_model.fit(inputs, np.zeros([inputs[0].shape[0], 3]), batch_size=200, epochs=1, verbose=False)
data_cur.append(network.predict(data))

plt.scatter(data_cur[-1][:, 0], data_cur[-1][:, 1], c=labels_cur[-1], cmap = cm)
plt.show()

### Merge labels

In [ ]:
_, _, W = affinity_computation(data_cur[-1], n_neighbors_pts)
A_us, A_s = compute_cluster_affinity(W, labels_cur_table)

nClusters = len(labels_cur_table)
print(nClusters)
unfold_iter = np.ceil(nClusters * unfold_rate)
unfold_iter_max = nClusters - n_clusters_target
n_iter = int(np.min([unfold_iter, unfold_iter_max]))
if n_iter > 0:
    labels_cur_table = run_agglomerative_clustering(A_us, A_s, labels_cur_table, n_iter, n_neighbors_clu)
labels_cur.append(convert_labels_tab(labels_cur_table))
nClusters = len(labels_cur_table)
print(nClusters)

plt.scatter(data_cur[-1][:, 0], data_cur[-1][:, 1], c=labels_cur[-1], cmap = cm)
plt.show()

### Train network

In [ ]:
for i in range(n_epochs):
    a, p, n = make_triplets(labels_cur[-1], n_clusters, n_neg)
    inputs = [data[a], data[p], data[n]]
    trainable_model.fit(inputs, np.zeros([inputs[0].shape[0], 3]), batch_size=200, epochs=1, verbose=False)
data_cur.append(network.predict(data))

plt.scatter(data_cur[-1][:, 0], data_cur[-1][:, 1], c=labels_cur[-1], cmap = cm)
plt.show()

### Merge labels

In [ ]:
_, _, W = affinity_computation(data_cur[-1], n_neighbors_pts)
A_us, A_s = compute_cluster_affinity(W, labels_cur_table)

nClusters = len(labels_cur_table)
print(nClusters)
unfold_iter = np.ceil(nClusters * unfold_rate)
unfold_iter_max = nClusters - n_clusters_target
n_iter = int(np.min([unfold_iter, unfold_iter_max]))
if n_iter > 0:
    labels_cur_table = run_agglomerative_clustering(A_us, A_s, labels_cur_table, n_iter, n_neighbors_clu)
labels_cur.append(convert_labels_tab(labels_cur_table))
nClusters = len(labels_cur_table)
print(nClusters)

plt.scatter(data_cur[-1][:, 0], data_cur[-1][:, 1], c=labels_cur[-1], cmap = cm)
plt.show()

### Train network

In [ ]:
for i in range(n_epochs):
    a, p, n = make_triplets(labels_cur[-1], n_clusters, n_neg)
    inputs = [data[a], data[p], data[n]]
    trainable_model.fit(inputs, np.zeros([inputs[0].shape[0], 3]), batch_size=200, epochs=1, verbose=False)
data_cur.append(network.predict(data))

plt.scatter(data_cur[-1][:, 0], data_cur[-1][:, 1], c=labels_cur[-1], cmap = cm)
plt.show()

### Final Ground truth

In [ ]:
plt.scatter(data_cur[-1][:, 0], data_cur[-1][:, 1], c=labels, cmap = cm)
plt.show()

In [ ]:
agg3 = AgglomerativeClustering(3)
clu_agg = agg3.fit_predict(data)

print(NMI(labels_cur[-1], labels))
print(NMI(labels, clu_agg))